In [15]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine


In [16]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')
user = os.getenv('user')
password = os.getenv('password')


In [17]:
csv_file_path = '../data/data_processed/MunE19_c_mod_municipios.csv'

In [18]:
table_name = 'municipios_pruebas_3'
primary_key_column = 'index'

In [19]:
try:
    # Connect to the MySQL database
    connection = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        port=port,
        database=database
    )
    
    if connection.is_connected():
        print("✅ Successfully connected to the database")

        df = pd.read_csv(csv_file_path, sep=',', encoding='utf-8')
        print(df.head())
        print(df.info())

        # Handle NaN values in the DataFrame
        df = df.where(pd.notnull(df), None)  # Replace NaN with None (NULL in MySQL)

        # Create a cursor and execute the query
        cursor = connection.cursor()

        # Create table if it does not exist
        # Generate a CREATE TABLE statement based on the DataFrame columns
        columns = ', '.join(f"`{col}` VARCHAR(255)" for col in df.columns)
        create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns}, PRIMARY KEY (`{primary_key_column}`))"
        cursor.execute(create_table_query)
        print(f"🛠️ Table '{table_name}' created or already exists.")

        # Insert data into the table
        try:
            print(f"ℹ️ Inserting data into '{table_name}'...")

            # Iterate over each row in the DataFrame
            for i, row in enumerate(df.itertuples(index=False), 1):
                # Generate the INSERT INTO query with properly escaped column names
                insert_query = f"INSERT INTO `{table_name}` (`{'`, `'.join(df.columns)}`) VALUES ({', '.join(['%s']*len(df.columns))})"
                
                # Execute the query with the values from the current row
                cursor.execute(insert_query, tuple(row))
                
                # Print progress message
                if i % 10 == 0:
                    print(f"Inserted {i} rows...")

            # Commit the changes
            connection.commit()
            print(f"✨ Data successfully inserted into '{table_name}'")

        except Error as e:
            # Handle any errors that may occur during insertion
            print("❌ Error inserting data:", e)

except Error as e:
    print("❌ An error occurred:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("🔒 MySQL connection is closed")

✅ Successfully connected to the database
   index           TH  Cod Comarca  Cod Municipio               Ámbito
0      0  Araba-Alava            1             59      VITORIA-GASTEIZ
1      1  Araba-Alava            2              1     ALEGRÍA-DULANTZI
2      2  Araba-Alava            2              3              ARAMAIO
3      3  Araba-Alava            2              6              ARMIÑÓN
4      4  Araba-Alava            2              8  ARRATZUA-UBARRUNDIA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          254 non-null    int64 
 1   TH             254 non-null    object
 2   Cod Comarca    254 non-null    int64 
 3   Cod Municipio  254 non-null    int64 
 4   Ámbito         254 non-null    object
dtypes: int64(3), object(2)
memory usage: 10.0+ KB
None
🛠️ Table 'municipios_pruebas_3' created or already exists.
ℹ️ Inserting 

In [20]:
connection_str = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

try:
    # Conectar a la base de datos MySQL con SQLAlchemy
    engine = create_engine(connection_str)
    connection = engine.connect()

    print("✅ Conexión exitosa a la base de datos")

    # Consultar los datos de la tabla y cargarlos en un DataFrame usando SQLAlchemy
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, con=engine)

    # Mostrar los primeros 5 registros para verificar
    print(f"🔍 Primeros 5 registros de la tabla '{table_name}':")
    print(df.head())
    print(df.info())

except mysql.connector.Error as e:
    print(f"❌ Error al conectar a la base de datos: {e}")

finally:
    # Cerrar la conexión a la base de datos
    if 'connection' in locals() and connection is not None:
        connection.close()
        print("🔒 Conexión MySQL cerrada")

✅ Conexión exitosa a la base de datos
🔍 Primeros 5 registros de la tabla 'municipios_pruebas_3':
  index           TH Cod Comarca Cod Municipio            Ámbito
0     0  Araba-Alava           1            59   VITORIA-GASTEIZ
1     1  Araba-Alava           2             1  ALEGRÍA-DULANTZI
2    10  Araba-Alava           2            17   CAMPEZO/KANPEZU
3   100      Bizkaia           3            91           ATXONDO
4   101      Bizkaia           3            92             BEDIA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          254 non-null    object
 1   TH             254 non-null    object
 2   Cod Comarca    254 non-null    object
 3   Cod Municipio  254 non-null    object
 4   Ámbito         254 non-null    object
dtypes: object(5)
memory usage: 10.0+ KB
None
🔒 Conexión MySQL cerrada
